# Découpage définitif des données au format AutoML

pour le format AutoML cf. la page https://github.com/madclam/m2aic2019/blob/master/Starting_Kit_M2info.pdf

- extraire les données de Magali
- les enrichir avec SMOTE
- passer au format AutoML (train, valid, test), en découpant de manière à ce que les classes soient équilibrées à chaque fois

### PLAN

- 1) chargement des données clean et génération du dataset global (1000 de chaque classe avec SMOTE)
- 2) découpage auto_ML global (TOUTES nos données): train, valid, test (800,100,100 pour chaque classe)
- 3) découpage sample pour starting_kit dans le TRAIN (!!!) précédent

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, log_loss
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import ADASYN, SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD

# Chargement des données clean

on charge les données et on utilise SMOTE pour créer l'ensemble "global" de données avec lequel on va travailler

In [17]:
# Chargement des données, clean_data étant les données nettoyées des NaN trop nombreux
df_data = pd.read_csv("clean_data.csv")
print("df_data shape:", df_data.shape)
df_metadata = pd.read_csv("metadata.csv")
print("df_metadata shape:", df_metadata.shape)
df_metadata[:5]

df_data shape: (20103, 687)
df_metadata shape: (685, 14)


,dmprocr_ID,indiv,sample,trscr,cnv,meth,gender,days_to_birth,tumor_stage,da,fut,age_diag,days_to_death,tissue_status
0,97-7552-01,97-7552,1,1,1,1,male,-25578.0,stage ib,alive,1932.0,25578.0,NaN,patho
1,44-7671-01,44-7671,1,0,1,1,male,-23538.0,stage ib,alive,889.0,23538.0,NaN,patho
2,86-7953-01,86-7953,1,1,1,1,female,-25315.0,stage ia,alive,997.0,25315.0,NaN,patho
3,L4-A4E5-01,L4-A4E5,1,1,1,1,female,-17680.0,stage i,alive,578.0,17680.0,NaN,patho
4,NJ-A4YP-01,NJ-A4YP,1,1,1,1,male,-19106.0,stage ib,alive,50.0,19106.0,NaN,patho


In [18]:
# Convertir les données en ndarray et supprimer les colonnes inutiles
D = df_data.loc[:, ~df_data.columns.str.contains('^Unnamed')].values
D = D.T

print(type(D))
print(D.shape)

# Générer les labels en fonction d'une colonne choisie
status = pd.Series(df_metadata["tissue_status"].values)
stage = pd.Series(df_metadata["tumor_stage"].values)

labelsBinary, valuesBinary = pd.factorize(status)
labelsStages, valuesStages = pd.factorize(stage)

yBinary = labelsBinary
yStage = labelsStages

print("labelsBinary :", labelsBinary)
print("valuesBinary :", valuesBinary)

print("labelsStages :", labelsStages)
print("valuesStages :", valuesStages)

<class 'numpy.ndarray'>
(685, 20103)
labelsBinary : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0

In [19]:
# IL FAUT ARRIVER À 300 MB après avoir agrandi le nombre d'observations
# ===>>> couper les features
# selection des k best features grâce au test chi2
chi2_selector = SelectKBest(chi2, k=1000)
D = chi2_selector.fit_transform(D, labelsBinary)
print(D.shape)

(685, 1000)


In [20]:
# 6 occurences nécessaire pour Smote ou Adasyn, 
# on enlève la classe 8 et on duplique une occurence de 2 pour passer à 6 (min SMOTE) 

X_train, X_test, y_train, y_test = train_test_split(D, yStage, test_size=0.2, random_state=42)
# stratify=y, mais une classe avec 1 occurences,on vire cette classe?

recounted = Counter(y_train)
print(recounted)

X_train_sans_8 = X_train[np.where(y_train!=8)]
X_2 = X_train[np.where(y_train==2)][0]
print(X_train_sans_8.shape)
print(X_2.shape)

# dédoublement d'un exemple de la classe 2
X_train_sans_8_double_2 = np.vstack([X_train_sans_8,X_2])
print(X_train_sans_8_double_2.shape)

# dédoublement d'un label de la classe 2
y_train_sans_8 = y_train[np.where(y_train!=8)]
print(y_train_sans_8)
y_train_sans_8 = np.append(y_train_sans_8, 2)
print(y_train_sans_8)

# retrait des instances potentielles de la classe 8 trop petite dans le test
X_test = X_test[np.where(y_test!=8)]
y_test = y_test[np.where(y_test!=8)]

Counter({0: 141, 1: 131, 5: 68, 3: 63, -1: 49, 7: 42, 4: 26, 9: 13, 6: 9, 2: 5, 8: 1})
(547, 1000)
(1000,)
(548, 1000)
[ 0  0  7  1  7  0  5  3  4 -1  0  0  5  0  7  0  6 -1  1  3  0  0  1  1
  3 -1  1  5  1  5  0  3  0  4  7  7  3  3  0  0  1  0  0  0  4 -1 -1 -1
  1  0  1  0 -1  0  9  6  5  0  0  0 -1  3  1  7  0  0  5  0  1  1  1  1
 -1  5  4  3  4  3 -1  0  7  0  5  1  1  6  1  9  1  0  7  5  1  1  0  5
  7  0  5  0  3  0  3  0  0  6  7  0  1  1  5  3  7  0  1  5  5  1  1  7
  0  1  0 -1 -1  0  1  0  0 -1  9  0 -1  0  3  1  0  7  5  1  3 -1  1  1
  7  0  1  1  0  1  1  6  5  7  9  1  1  0  7  3  1  1  0  7  4  4  3  5
  5  3  5 -1 -1  0  7  3  0  1  3  6  0  9  0  6  0  4  0  3 -1  1  7 -1
  3  1  4  0  1  4  1  1  1  5  0 -1  9  1  0  1  1 -1  5  5  0 -1  3  4
  0  1  7 -1  1  1  1  1  7  3  7  0  0  4  5  5  0  3  2  5  7 -1  1  7
  0  5  1  0  1  1  0  1  3  1  5  1 -1  0  3  0  7  1  5  3  9 -1  0  5
  1  1  5  1  9  9  3  3  0  0  1  1 -1 -1  5  1  1  1  0  0  0  2  0  0
  6  

In [21]:
X_train = X_train_sans_8_double_2
y_train = y_train_sans_8

# nombre d'occurences désirées par classe 
# # on a enlevé la classe 8 car qu'une occurence 
# impossible d'appliquer SMOTE ou ADASYN et génération d'une population à partir
# d'un seul exemple est absurde

dict= {0: 1000, 1: 1000, 5: 1000, 3: 1000, -1: 1000, 7: 1000, 4: 1000, 9: 1000, 6: 1000, 2: 1000}  
smote = SMOTE(random_state=42, sampling_strategy=dict)

X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
y_resampled = np.array([y_1 if y_1 != -1 else 8 for y_1 in y_resampled])
print(set(y_resampled))
print(X_resampled.shape)
print(y_resampled.shape)

/home/mbauw/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (1000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 141)
  n_samples_majority))
/home/mbauw/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (1000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 141)
  n_samples_majority))
/home/mbauw/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (1000) in class 5 will be larger than the number of samples in the majority class (class #0 -> 141)
  n_samples_majority))
/home/mbauw/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (1000) in class 3 will be larger than the number of samples in the 

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
(10000, 1000)
(10000,)


# Découpage auto_ML global (TOUTES nos données): train, valid, test

chaque ensemble (train, valid, test) doit être équilibré

In [55]:
# on récupère un data_frame de chaque classe, dans lesquelles on va piocher pour redéfinir chaque ensemble

X_resampled_df = pd.DataFrame(X_resampled)
y_resampled_df = pd.DataFrame(y_resampled)
y_resampled_df = y_resampled_df.rename(columns={0: 'label'})

X_and_y = pd.concat([X_resampled_df, y_resampled_df], axis=1, sort=False)
#X_and_y.head(n=2)
print(X_and_y.shape)

(10000, 1001)


In [37]:
np.unique(X_and_y['label'].values) # vérifie la disparition de la classe -1

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [62]:
# définition d'un dataframe par classe pour découper par classe
# et obtenir des train/valid/test équilibrés

X_and_y_class0 = X_and_y[X_and_y["label"] == 0]
X_and_y_class1 = X_and_y[X_and_y["label"] == 1]
X_and_y_class2 = X_and_y[X_and_y["label"] == 2]
X_and_y_class3 = X_and_y[X_and_y["label"] == 3]
X_and_y_class4 = X_and_y[X_and_y["label"] == 4]
X_and_y_class5 = X_and_y[X_and_y["label"] == 5]
X_and_y_class6 = X_and_y[X_and_y["label"] == 6]
X_and_y_class7 = X_and_y[X_and_y["label"] == 7]
X_and_y_class8 = X_and_y[X_and_y["label"] == 8]
X_and_y_class9 = X_and_y[X_and_y["label"] == 9]

### TRAIN global de taille 800 (dans lequel on prendra tous les sets du starting kit)

In [66]:
X_and_y_class0_train = X_and_y_class0[:800]
print(X_and_y_class0_train.shape)

X_and_y_class1_train = X_and_y_class1[:800]
X_and_y_class2_train = X_and_y_class2[:800]
X_and_y_class3_train = X_and_y_class3[:800]
X_and_y_class4_train = X_and_y_class4[:800]
X_and_y_class5_train = X_and_y_class5[:800]
X_and_y_class6_train = X_and_y_class6[:800]
X_and_y_class7_train = X_and_y_class7[:800]
X_and_y_class8_train = X_and_y_class8[:800]
X_and_y_class9_train = X_and_y_class9[:800]

# TRAIN global par concaténations des 800 premiers de chaque classe pour obtenir un train équilibré
X_and_y_train = pd.concat([X_and_y_class0_train,X_and_y_class1_train, X_and_y_class2_train,X_and_y_class3_train,X_and_y_class4_train,X_and_y_class5_train,X_and_y_class6_train,X_and_y_class7_train,X_and_y_class8_train,X_and_y_class9_train], axis=0, sort=False)
print(X_and_y_train.shape)

(800, 1001)
(8000, 1001)


### VALID global de taille 100 (dans lequel on ne prendra RIEN pour le starting kit)

In [69]:
X_and_y_class0_valid = X_and_y_class0[800:900]
X_and_y_class1_valid = X_and_y_class1[800:900]
X_and_y_class2_valid = X_and_y_class2[800:900]
X_and_y_class3_valid = X_and_y_class3[800:900]
X_and_y_class4_valid = X_and_y_class4[800:900]
X_and_y_class5_valid = X_and_y_class5[800:900]
X_and_y_class6_valid = X_and_y_class6[800:900]
X_and_y_class7_valid = X_and_y_class7[800:900]
X_and_y_class8_valid = X_and_y_class8[800:900]
X_and_y_class9_valid = X_and_y_class9[800:900]

# TRAIN global par concaténations des 800 premiers de chaque classe pour obtenir un train équilibré
X_and_y_valid = pd.concat([X_and_y_class0_valid,X_and_y_class1_valid, X_and_y_class2_valid,X_and_y_class3_valid,X_and_y_class4_valid,X_and_y_class5_valid,X_and_y_class6_valid,X_and_y_class7_valid,X_and_y_class8_valid,X_and_y_class9_valid], axis=0, sort=False)
print(X_and_y_valid.shape)

(1000, 1001)


### TEST global de taille 100 (dans lequel on ne prendra RIEN pour le starting kit)

In [1]:
# TO DO

# Découpage sample pour starting_kit dans le train précédent (sample train/valid/test provenant du train global !!!)


ATTENTION CI DESSOUS COPIE COLLE NON ADAPTE (necessaire ?) de Sid Ali

In [ ]:
with open('../starting_kit/sample_data/hadaca_feat1.name', 'w') as f:
    for i in range(X_resampled.shape[1]):
        f.write('methyl_{}\n'.format(i))

with open('../starting_kit/sample_data/hadaca_train.data', 'w') as f:
    for x in X_resampled[50:100]:
        for feat in x:
            f.write('{} '.format(feat))

        f.write('\n')

with open('../starting_kit/sample_data/hadaca_test.data', 'w') as f:
    for x in X_resampled[:50]:
        for feat in x:
            f.write('{} '.format(feat))

        f.write('\n')


with open('../starting_kit/sample_data/hadaca_valid.data', 'w') as f:
    for x in X_resampled[100:150]:
        for feat in x:
            f.write('{} '.format(feat))

        f.write('\n')

with open('../starting_kit/sample_data/hadaca_train.solution', 'w') as f:
    for x in y_resampled[50:100]:
        f.write('{}'.format(x))

        f.write('\n')

with open('../starting_kit/sample_data/hadaca_test.solution', 'w') as f:
    for x in y_resampled[:50]:
        f.write('{}'.format(x))

        f.write('\n')


with open('../starting_kit/sample_data/hadaca_valid.solution', 'w') as f:
    for x in y_resampled[100:150]:
        f.write('{}'.format(x))

        f.write('\n')